# Red Neuronal de reconocimento facial

### El proyecto consiste en una red neuronal que recibe imagenes de entrada para entrenamiento (donde se indica los nombres de las personas de cada imagen) de la misma red, posteriormente se introducen imagenes de prueba, sin nombres, para que la red neuronal indique el nombre de la persona

#### Conozcan a Riho takada:
![title](RihoTakadaFULL.jpg)

#### Y a Rui Kiriyama:
![Rui](RuiKiriyamaFULL.jpg)
(Si... un poco pervertido el asunto, pero #YOLO)

#### Quienes serán nuestras modelos, se entrenará la red con una variedad de imagenes de ellas, incluyendo el nombre para que esta aprenda a hacer el reconocimiento facial.
#### Por simplicidad, nos ahorramos el paso de detección facial (es decir, de una foto completa, que el algoritmo encuentre el rosto) y solamente pasaremos el rostro para hacer el entrenamiento. Si sobra tiempo, realizaremos también la detección facial
#### Las imagenes se pasarán en un formato .png con un tamaño de 300 x 300 px
### ACTUALIZACIÓN:
#### Las imagenes se van a pasar en formato .png con tamaño de 30 x 30 px, inicialmente se pasará cada pixel como entrada a la red neuronal

# Creación de la red 

## (Primer objetivo propuesto)

In [25]:
import numpy as np

#La red neuronal 
#Primero hay que calcular feed-forward
#Después hay que calcular el error
#Ya calculado el error se realiza el back-propagation
#Se actualizan los pesos        

class neuralnet:
    def __init__(self, inpn):
        self.inputneurons=inpn #Numero de neuronas de entrada
        self.outputneurons=2 #Numero de neuronas de salida, como ya sé que van a ser sólo dos, de una vez la fijo en 2
        
        #Iniciamos las capas ocultas como un arreglo de zeros
        #Iniciamos los pesos
        self.w1=np.random.randn(inpn, inpn)
        self.w2=np.random.randn(inpn, inpn)
        self.w3=np.random.randn(inpn, inpn)
        self.w4=np.random.randn(inpn, 2)
        
    #Función sigmoide
    def sigmoid(self, x, d):
        """
        Función sigmoide
        Entrada: variable flotante X (valor inicial de una neurona de la capa oculta), booleano D para definir si queremos la derivada o la función original
        Salida: el resultado de aplicar a X la función sigmoide (o su derivada)
        """
        if (d == True):
            return x*(1-x)
        else:
            return 1/(1+np.exp(-x))
    
    #Para entrenar la red, recibe un ejemplo de entrenamiento y el número de epocas de entrenamiento deseadas, de preferencia arriba de 10000
    #UTILIZAREMOS 3 CAPAS OCULTAS, COMO AÚN NO EXTRAEMOS LAS CARACTERÍSTICAS DE LAS IMAGENES, ESTO SE VA A DESCONTROLAR FEO
    def train(self, inputarr, Y, epoch):
        #PRIMERA PRUEBA, QUE CADA PIXEL DE LA ENTRADA SEA UNA NEURONA DE ENTRADA
        X=inputarr.flatten().reshape(1,self.inputneurons)
        graphvar=[]
        for e in range(epoch):
            #Feed-forward
            h0 = self.sigmoid(np.dot(X, self.w1), False)
            h1 = self.sigmoid(np.dot(h0, self.w2), False)
            h2 = self.sigmoid(np.dot(h1, self.w3), False)
            Outl = self.sigmoid(np.dot(h2, self.w4), False)
            
            #Calculo de error
            error_calc= Y - Outl
            
            #BonitoDebug
            if (e%1000==0):
                graphvar.append(np.mean(np.abs(error_calc)))
                print("Error medio: " + str(np.mean(np.abs(error_calc))))
                
            #Back-Propagation
            outDelta = error_calc*self.sigmoid(Outl, True)            
            h2err = outDelta.dot(self.w4.T)
            h2Delta = h2err * self.sigmoid(h2, True)
            h1err = h2Delta.dot(self.w3.T)
            h1Delta = h1err * self.sigmoid(h1, True)
            h0err = h1Delta.dot(self.w2.T)
            h0Delta = h0err * self.sigmoid(h0, True)
            
            #Actualización de pesos
            self.w1+=X.T.dot(h0Delta)
            self.w2+=h0.T.dot(h1Delta)
            self.w3+=h1.T.dot(h2Delta)
            self.w4+=h2.T.dot(outDelta)
        
        #Para efectos prácticos graficaremos esta babosada, a ver si está aprendiendo o no
        graphx=[i for i in range(len(graphvar))]
        plt.plot(graphx, graphvar)
        
    def test(self, testarr):
        X=testarr.flatten().reshape(1,self.inputneurons)
        l0 = self.sigmoid(np.dot(X, self.w1), False)
        l1 = self.sigmoid(np.dot(l0, self.w2), False)
        l2 = self.sigmoid(np.dot(l1, self.w3), False)
        self.pred_o = self.sigmoid(np.dot(l2, self.w4), False)
        if (self.pred_o[0][0]>self.pred_o[0][1]):
            print("Predicción: Rui Kiriyama")
        else:
            print("Predicción: Takada Riho")
        

###### Como hacer que Python abra imagenes sin morir en el intento

In [29]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def Abrir(imtype, nom):
    """
    Función abrir: Se utiliza para abrir una imagen dado el nombre del archivo (string)
    Entrada: imtype variable para definir si queremos abrir una imagen de entramiento (train) o de prueba (test)
    Salida: Un objeto imagen matplotlib
    """
    if (imtype=='train'):
        prefix='TrainSamples/'
    elif (imtype=='test'):
        prefix='TestSamples/'
    else:
        print("Error, prefijo incorrecto: debe ser 'train' o 'test'")
    return mpimg.imread(prefix+nom+'.png')

#Probemos la wea anterior
#Algo flojo, pero para ya no poner las comillas sencillas
train='train'
test='test'
img = Abrir(train, '001')
img2 = Abrir(train, '101')

In [30]:
'''
    Ya que vimos que si funciona esto, tenemos que asociar el nombre a cada imagen, lo ideal sería crear objetos de entrenamiento
    donde podamos poner absolutamente TODA la información relevante de las imagenes, como nombre, características, y la misma imagen obviamente
'''

#TODO: Extraer las características 
class TrainingElem:
    #Acá irán las demás features
    def __init__(self, name, tipo, path):
        self.name=name
        self.img=Abrir(tipo, path)

#### DE MOMENTO
##### Hicimos que la entrada sea forzosa de 30 x 30 pixeles, para la primera prueba entrenaremos la red pasando cada pixel como valor de entrada es decir, tendremos 900 neuronas de entrad; además usaremos 3 capas ocultas, para que no sea tan simple.
##### Como tenemos 2 modelos, sólo habrá 2 neuronas de salida: 1.- Rui Kiriyama : 2.- Riho Takada, en dado caso que la foto de la imagen correspondiera a una, se espera que la neurona de salida correspondiente tenga un 1 y la otra neurona un 0, es de aquí donde comenzaremos a corregir el error y modificar los pesos para entrenar la red

In [31]:
#Probemos un ejemplo, a ver si sirve la red neuronal
#size=img.shape[0]*img.shape[1]
#red=neuralnet(size)
#Para la Y = Y[0] Rui Kiriyama, Y[1] = Riho Takada
#ADVERTENCIA, COMO YA TIENE 3 CAPAS OCULTAS Y AÚN NO EXTRAEMOS CARACTERÍSTICAS DE LA IMAGEN, EL PROCESO ES MUY TARDADO
#red.train(img[:,:,0], [1,0], 10000)
#red.train(img2[:,:,0], [0,1], 10000)
#red.test(img2[:,:,0])
#imgplot = plt.imshow(img2)
#print(red.pred_o)

# Extracción de características

### (Segundo objetivo propuesto)

Hasta ahorita ya cumplimos el primer objetivo (crear la red neuronal), la cuál se instancia como un objeto y cuenta con sus métodos para entrenar y para probar con simplemente pasarle una imagen, la cuál se "rompe" de momento en pixeles los cuáles "alimentan" la red neuronal como valores de entrada.

Sin embargo, este método no es eficiente ni óptimo, entrenar la red neuronal con tan solo 1 ejemplo de entrenamiento tarda mucho tiempo, esto se vuelve muy torpe al querer introducir varios ejemplos de entrenamiento, además que la salida que "predice" no es muy acertada.

Para esto (tras realizar un poco de investigación) utilizaremos "Eigenfaces"

## Eigenfaces

El concepto consiste en un conjunto de eigenvectores (vectorer característicos de una transformación lineal -> un poco de álgebra lineal), los cuales forman un conjunto base de todas las imágenes utilizadas para construir una matriz de covarianza (a su vez, estos se derivan de esta, de la distribución de probabilidad del espacio vectorial de imágenes de rostros)

### Cómo generarlas

Un conjunto de eigenfaces se puede generar aplicando un proceso matemático llamado "Análisis de componente principal" (PCA) en un conjunto de imágenes. Por ejemplo: un rostro se puede generar del "rostro promedio" + 10%, 55%,  -3% de las eigenfaces 1, 2 y 3 respectivamente.